<a href="https://colab.research.google.com/github/DoItSon/Dacon/blob/main/%EA%B1%B4%EC%84%A4%EA%B8%B0%EA%B3%84%20%EC%98%A4%EC%9D%BC%20%EC%83%81%ED%83%9C%20%EB%B6%84%EB%A5%98%20AI%20%EA%B2%BD%EC%A7%84%EB%8C%80%ED%9A%8C/%EB%A8%B8%EC%8B%A0%EB%9F%AC%EB%8B%9D_%ED%95%B4%EB%B3%B4%EA%B8%B0(0_2)!.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split

import os
import pandas as pd
import numpy as np
from tqdm.auto import tqdm
import random

from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.metrics import f1_score

from lightgbm import LGBMClassifier,plot_importance

from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

from sklearn.metrics import explained_variance_score, mean_absolute_error

import warnings
warnings.filterwarnings(action='ignore')

In [ ]:
!pip install optuna

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 348 kB 4.4 MB/s 
     |████████████████████████████████| 209 kB 13.4 MB/s 
     |████████████████████████████████| 81 kB 1.4 MB/s 
     |████████████████████████████████| 78 kB 1.2 MB/s 
     |████████████████████████████████| 147 kB 20.3 MB/s 
     |████████████████████████████████| 50 kB 2.0 MB/s 
     |████████████████████████████████| 112 kB 8.7 MB/s 
  Created wheel for pyperclip: filename=pyperclip-1.8.2-py3-none-any.whl size=11136 sha256=b78e7fb7736d516c19b17ea127e541932606d5340ccc1e9e2ff72bd4f35d9d33
  Stored in directory: /root/.cache/pip/wheels/7f/1a/65/84ff8c386bec21fca6d220ea1f5498a0367883a78dd5ba6122
Successfully built pyperclip


In [ ]:
import optuna

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
DATA_PATH = "/content/drive/MyDrive/"

In [ ]:
train = pd.read_csv(f'{DATA_PATH}train.csv')
test = pd.read_csv(f'{DATA_PATH}test.csv')

In [ ]:
categorical_features_enc = ['COMPONENT_ARBITRARY_COMPONENT1',
       'COMPONENT_ARBITRARY_COMPONENT2', 'COMPONENT_ARBITRARY_COMPONENT3',
       'COMPONENT_ARBITRARY_COMPONENT4', 'YEAR_2007', 'YEAR_2008', 'YEAR_2009',
       'YEAR_2010', 'YEAR_2011', 'YEAR_2012', 'YEAR_2013', 'YEAR_2014',
       'YEAR_2015', 'YEAR_2016', 'YEAR_2017', 'YEAR_2018', 'YEAR_2019',
       'YEAR_2020', 'YEAR_2021', 'YEAR_2022']

## 전처리

In [ ]:
CFG = {
    'n_splits':5,
    'SEED':42
}

- 내가 직접한 결측치

- 머신러닝

In [ ]:
cols = ['CD','FOPTIMETHGLY',"FUEL",'K','U100','U75','U50',"FH2O","FNOX","FOXID","FSO4","FTBN","SOOTPERCENTAGE","U25","U20","U14","U6","U4","V100"]

In [ ]:
for i in cols:
    imputer = IterativeImputer(estimator =  lgb.LGBMRegressor(),random_state=CFG["SEED"])
    train[i] = imputer.fit_transform(train[[i]])

In [ ]:
train.isnull().sum().sum()

In [ ]:
train.columns

## 피처추가
- 점수가 떨어져서 피처추가 안함!

## 데이터 분리

In [ ]:
train_X  = train.drop(['ID', 'Y_LABEL'], axis = 1) # 정답값 제외
target  = train['Y_LABEL'] # 정답

test = test.drop(['ID'], axis = 1) # 추론 데이터

In [ ]:
train_X

# 원핫인코딩

In [ ]:
enc = OneHotEncoder()

tmp = pd.DataFrame(
    enc.fit_transform(train_X[["COMPONENT_ARBITRARY",'YEAR']]).toarray(),
    columns = enc.get_feature_names_out()
)
train_X = pd.concat([train_X,tmp],axis=1).drop(columns = ["COMPONENT_ARBITRARY",'YEAR'])

tmp1 = pd.DataFrame(
    enc.transform(test[["COMPONENT_ARBITRARY",'YEAR']]).toarray(),
    columns = enc.get_feature_names_out()
)
test = pd.concat([test,tmp1],axis=1).drop(columns = ["COMPONENT_ARBITRARY",'YEAR'])

In [ ]:
train_X.shape,target.shape,test.shape

# 스케일링

In [ ]:
def get_values(value):
    return value.values.reshape(-1, 1)

for col in train_X.columns:
    if col not in categorical_features_enc:
        scaler = StandardScaler()
        train_X[col] = scaler.fit_transform(get_values(train_X[col]))
        if col in test.columns:
            test[col] = scaler.transform(get_values(test[col]))

## Classifier

In [ ]:
# def objective_1(trial):
#     hp = {
#         "max_depth" : trial.suggest_int("max_depth",2,15), # 찾을 범위의 최소값과 최대값
#         "min_samples_split" : trial.suggest_int("min_samples_split",2,10),
#         "criterion" : trial.suggest_categorical("criterion",["gini","entropy"]),
#         "max_leaf_nodes" : trial.suggest_int("max_leaf_nodes",5,10),
#         "n_estimators" : trial.suggest_int("n_estimators",10,500),
#         "learning_rate" : trial.suggest_float("learning_rate",0.01,0.1)
#     }
#     cv = KFold(n_splits=5,shuffle=True,random_state=CFG['SEED'])
#     model = LGBMClassifier(random_state=CFG['SEED'],**hp)

#     scores = cross_val_score(model, train_X, target,cv=cv, scoring="f1_macro") # cv=cv 지움!
#     return np.mean(scores)

In [ ]:
# sampler = optuna.samplers.TPESampler(seed = CFG['SEED']) 

# study_1 = optuna.create_study(
#     direction= "maximize", 
#     sampler = sampler
# )

# study_1.optimize(objective_1,n_trials=50)

In [ ]:
# print("Best Score:",study_1.best_value) 
# print("Best hp", study_1.best_params)

In [ ]:
# study_1.best_params # 따로 적어놓기

In [ ]:
hp = {'max_depth': 15,
 'min_samples_split': 2,
 'criterion': 'entropy',
 'max_leaf_nodes': 10,
 'n_estimators': 495,
 'learning_rate': 0.0943448646911196}

In [ ]:
cv = KFold(n_splits=5,shuffle=True,random_state=CFG["SEED"])

In [ ]:
score_list = []

model = LGBMClassifier(random_state=CFG['SEED'],**hp) # use_missing=False 결측치 옵션
model.fit(train_X,target , eval_set = [(train_X,target)],early_stopping_rounds=50) 

pred = model.predict_proba(train_X)[:,1] 

score = cross_val_score(model,train_X,target,cv = cv ,scoring='f1_macro',n_jobs = -1)
score_list.append(score) 

## 특성 중요도 확인하기

In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize=(40,20))
plot_importance(model,ax=ax)
plt.show()

In [ ]:
np.mean(score_list)

- 확률!!!

In [ ]:
x_target = pred
x_target

## Regressor

- train의 부분 X의 예측값을 설명하는 Regressor

In [ ]:
test_stage_features = ['ANONYMOUS_1', 'ANONYMOUS_2', 'AG', 'CO', 'CR', 'CU', 'FE', 'H2O', 'MN',
       'MO', 'NI', 'PQINDEX', 'TI', 'V', 'V40', 'ZN',
       'COMPONENT_ARBITRARY_COMPONENT1', 'COMPONENT_ARBITRARY_COMPONENT2',
       'COMPONENT_ARBITRARY_COMPONENT3', 'COMPONENT_ARBITRARY_COMPONENT4',
       'YEAR_2007', 'YEAR_2008', 'YEAR_2009', 'YEAR_2010', 'YEAR_2011',
       'YEAR_2012', 'YEAR_2013', 'YEAR_2014', 'YEAR_2015', 'YEAR_2016',
       'YEAR_2017', 'YEAR_2018', 'YEAR_2019', 'YEAR_2020', 'YEAR_2021',
       'YEAR_2022']

In [ ]:
train_reg = train_X[test_stage_features]
train_reg.shape,x_target.shape

In [ ]:
# from lightgbm.sklearn import LGBMRegressor
# def objective_2(trial):
#     hp = {
#         "max_depth" : trial.suggest_int("max_depth",2,15), # 찾을 범위의 최소값과 최대값
#         "min_samples_split" : trial.suggest_int("min_samples_split",2,10),
#         "criterion" : trial.suggest_categorical("criterion",["gini","entropy"]),
#         "max_leaf_nodes" : trial.suggest_int("max_leaf_nodes",5,10),
#         "n_estimators" : trial.suggest_int("n_estimators",10,500),
#         "learning_rate" : trial.suggest_float("learning_rate",0.01,0.1)
#     }
#     cv = KFold(n_splits=5,shuffle=True,random_state=CFG['SEED'])
#     model = LGBMRegressor(random_state=CFG['SEED'],**hp)

#     scores = cross_val_score(model, train_reg, x_target,cv=cv ,scoring="neg_mean_absolute_error") # cv=cv 지움!
#     return np.mean(scores)

In [ ]:
# sampler = optuna.samplers.TPESampler(seed = CFG['SEED']) 

# study_1 = optuna.create_study(
#     direction= "maximize", 
#     sampler = sampler
# )

# study_1.optimize(objective_2,n_trials=50)

In [ ]:
# print("Best Score:",study_1.best_value) 
# print("Best hp", study_1.best_params)

In [ ]:
# study_1.best_params # 따로 적어놓기

In [ ]:
hp3 = {'max_depth': 15,
 'min_samples_split': 8,
 'criterion': 'gini',
 'max_leaf_nodes': 8,
 'n_estimators': 407,
 'learning_rate': 0.02281424026216031}

In [ ]:
score_list= []
model = lgb.LGBMRegressor(random_state=CFG['SEED'],**hp3)
model.fit(train_reg,x_target, eval_set = [(train_reg,x_target)],early_stopping_rounds=50) 

pred = model.predict(train_reg)

score = -1*cross_val_score(model, train_reg, pred,cv=cv ,scoring="neg_mean_absolute_error") # cv=cv 지움! # 내가 예측한 값과 실제값의 차이를 계산한 score
score_list.append(score)

In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize=(40,20))
plot_importance(model,ax=ax)
plt.show()

In [ ]:
np.mean(score_list)

### test의 shape과 맞춘 후 test에 적합

In [ ]:
pred = model.predict(test)

In [ ]:
y_pred = pred

In [ ]:
pred

# 산술평균 앙상블

In [ ]:
np.save(f"{DATA_PATH}/LGBM",pred)

In [ ]:
pred_1 = np.load(f"{DATA_PATH}/XGB.npy")
pred_2 = np.load(f"{DATA_PATH}/LGBM.npy")
pred_3 = np.load(f"{DATA_PATH}/CAT.npy")

In [ ]:
pred_list = pred_1, pred_1, pred_3

In [ ]:
pred = np.mean(pred_list,axis=0)
pred.shape

(6041,)

In [ ]:
y_pred = pred

In [ ]:
submit = pd.read_csv('/content/drive/MyDrive/sample_submission.csv')

In [ ]:
submit.Y_LABEL = pd.Series(np.where(y_pred > 0.2, 1, 0)) # 이것도 추후 조정해보기!

In [ ]:
submit

,ID,Y_LABEL
0,TEST_0000,0
1,TEST_0001,0
2,TEST_0002,0
3,TEST_0003,0
4,TEST_0004,0
...,...,...
6036,TEST_6036,0
6037,TEST_6037,0
6038,TEST_6038,0
6039,TEST_6039,0


In [ ]:
submit.Y_LABEL.value_counts()

0    5721
1     320
Name: Y_LABEL, dtype: int64

In [ ]:
submit.to_csv('/content/drive/MyDrive/submit.csv', index=False)

# 하드보팅

In [ ]:
voting1 = pd.read_csv(f'{DATA_PATH}LGBM (0.2).csv')
voting2 = pd.read_csv(f'{DATA_PATH}CAT(0.2).csv')
voting3 = pd.read_csv(f'{DATA_PATH}XGB 0.2.csv')

In [ ]:
target_dict = {'review1': voting1['Y_LABEL'], 'review2': voting2['Y_LABEL'], 'review3': voting3['Y_LABEL']}
df_ensemble = pd.DataFrame(target_dict)
df_ensemble

In [ ]:
from collections import Counter
def ensemble_data(data):
    # 1. 매열마다 모든 값을 Count, 가장 많은 빈도의 값을 추출.
    result_list = []
    for i in range(len(data)):
        frequent_value = data.iloc[i].value_counts().idxmax()
        result_list.append(frequent_value)
    return pd.DataFrame({'Y_LABEL':result_list})

In [ ]:
result = ensemble_data(df_ensemble)
result 

In [ ]:
result.Y_LABEL.value_counts()

In [ ]:
submit = pd.read_csv('/content/drive/MyDrive/sample_submission.csv')
submit["Y_LABEL"] = result
submit.to_csv('/content/drive/MyDrive/submit.csv', index=False)
submit

In [ ]:
result.Y_LABEL.value_counts()